# M-Dimensional Estimator For Mutual Information

This notebook presents an extension for computing the estimator of mutual information between 2 random variables to m random variables. Sklearn has a function for such a 2-dimensional problem called `mutual_info_regression`. The source code is modified such that we can take a $p$ dimensional design matrix and estimate the mutual information between $m$ predictors and the target. The estimator can be found in the following paper: https://journals.aps.org/pre/pdf/10.1103/PhysRevE.69.066138

## Mutual Information of Two Random Variables

$$I(X;Y) = \int \int p_{X, Y}(x, y) \log \frac{p_{X, Y}(x, y)}{p_X(x)p_Y(y)}$$

Notice that this takes the shape of the [Kullback-Leibler Divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence). Also recall that if $X$ and $Y$ are completely independent, then the joint probability distribution is the product of the marginals. In this case, indepedent continuous random variables yield a mutual information of 0, which can be observed above.

### A 2-Dimensional Estimator of Mutual Information

$$I(X, Y) = \psi(k) + \psi(N) - \langle \psi(n_{x_1 + 1}) + \psi(n_{y + 1})\rangle$$

Suppose $Z = (X, Y)$

where

- $\psi(x)$ is the digamma function

- $k$ is the number of nearest neighbors

- $m$ is the number if dimensions

- $N$ is the number of samples

- $n_{x_i}$ is the number of points $x_j$ strictly less than the radius, where the radius is the distance from $z_i$ to its $k^{th}$ neighbor. 

As shown in the paper, the authors propose an estimator from a nearest-neighbor approach which diverges from the traditional binning approach. They then briefly mention an extension for an M-dimensional estiamtor, which is given below.

### An M-Dimensional Estimator of Mutual Information

$$I(X_1, X_2,...,X_m) = \psi(k) + (m-1)\psi(N) - \langle \psi(n_{x_1}) + \psi(n_{x_2}) + ... + \psi(n_{x_m}) \rangle$$

Suppose $Z = (X_1, X_2,...,X_m)$

where

- $\psi(x)$ is the digamma function

- $k$ is the number of nearest neighbors

- $m$ is the number if dimensions

- $N$ is the number of samples

- $n_{x_i}$ is the number of points $x_j$ strictly less than the radius, where the radius is the distance from $z_i$ to its $k^{th}$ neighbor. 

## Implementation

The code below modifies the source code from sklearn's `mutual_info_regression` for a 2-dimensional estimator to correspond to the M-dimensional estimator given in the paper.

In [82]:
from itertools import combinations
import numpy as np
from sklearn.neighbors import NearestNeighbors, KDTree
import matplotlib.pyplot as plt
from functools import reduce
from scipy.special import digamma
import pandas as pd
from sklearn.feature_selection import mutual_info_regression

In [220]:
def compute_mi_cc(X, y, N, n_neighbors):

    m = X.shape[1]
    Xy = np.hstack((X, y))

    nn = NearestNeighbors(metric='chebyshev', n_neighbors=n_neighbors)

    nn.fit(Xy)
    radius = nn.kneighbors()[0]
    radius = np.nextafter(radius[:, -1], 0)

    n_i = []
    for i in range(Xy.shape[1]):
        
        Xy_i = Xy[ : , i].reshape((-1, 1))
        kd = KDTree(Xy_i, metric='chebyshev')
        n_i.append(kd.query_radius(Xy_i, radius, count_only=True, return_distance=False) - 1)

    digamma_mean = lambda x : np.mean(digamma(x + 1))
    mi = digamma(n_neighbors) + m * digamma(N) - sum(list(map(digamma_mean, n_i)))

    return max(0, mi)

In [221]:
def m_dim_mi(X, y, m = 2, n_neighbors = 3):
    
    p = X.shape[1]
    all_pairs = list(combinations(range(p), m))
    
    y = y.reshape((-1, 1))
    N = y.size
    
    mis = [compute_mi_cc(X[ : , pair], y, N, n_neighbors) for pair in all_pairs]
    
    return mis

### M-way MI

In [239]:
p = 5
n = 10000
X = np.random.normal(size = (n, p))
y = X[ : , 0] * X[ : , 1] + np.random.normal(scale = 0.1, size = n)

m = 2
mi_scores = m_dim_mi(X, y, m = m)
all_pairs = list(combinations(range(p), m)) 

indices = ['x' + str(pair[0]) + ' and x' + str(pair[1]) for pair in all_pairs]
pd.Series(mi_scores, name="MI Scores", index=indices)

x0 and x1    2.075973
x0 and x2    0.316282
x0 and x3    0.318014
x0 and x4    0.326788
x1 and x2    0.318247
x1 and x3    0.318675
x1 and x4    0.326551
x2 and x3    0.000718
x2 and x4    0.000000
x3 and x4    0.005980
Name: MI Scores, dtype: float64

### 1-way MI

In [240]:
y = y.reshape((-1, 1))
Xy = np.hstack((X, y))
df = pd.DataFrame(Xy)
df.columns = ['x' + str(i) for i in range(p)] + ['y']
y = df.pop('y')
discrete_features = df.dtypes == int

mi_scores = mutual_info_regression(df, y, discrete_features=discrete_features)
pd.Series(mi_scores, name="MI Scores", index=df.columns)

x0    0.331598
x1    0.327218
x2    0.000000
x3    0.002460
x4    0.011109
Name: MI Scores, dtype: float64